# 🏃 Analyse des Résultats et Performances des Athlètes

Ce notebook analyse les performances olympiques des athlètes à travers l'histoire.

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from database.connexion import get_connection

# Configuration pour de meilleurs graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [ ]:
# ============================================================
# 🔌 CONNEXION À LA BASE DE DONNÉES ET CHARGEMENT DES DONNÉES
# ============================================================
conn = get_connection()

# Vérifier les tables disponibles
cursor = conn.cursor()
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()
print("📋 Tables disponibles :")
for table in tables:
    print(f"   - {table[0]}")

# Essayer de charger la table athletes
try:
    athletes_df = pd.read_sql("SELECT * FROM athletes", conn)
    print(f"\n✅ Table 'athletes' chargée : {len(athletes_df)} enregistrements")
    has_athletes = True
except Exception as e:
    print(f"\n⚠️ Erreur chargement 'athletes': {e}")
    has_athletes = False

# Essayer de charger la table results
try:
    results_df = pd.read_sql("SELECT * FROM results", conn)
    print(f"✅ Table 'results' chargée : {len(results_df)} enregistrements")
    has_results = True
except Exception as e:
    print(f"⚠️ Erreur chargement 'results': {e}")
    has_results = False

conn.close()

# Choisir la table à analyser
if has_results and len(results_df) > 0:
    data_df = results_df
    data_source = "results"
    print(f"\n📊 Analyse de la table 'results' ({len(data_df)} enregistrements)")
elif has_athletes:
    data_df = athletes_df
    data_source = "athletes"
    print(f"\n📊 Analyse de la table 'athletes' ({len(data_df)} enregistrements)")
else:
    print("\n❌ Aucune donnée disponible pour l'analyse")
    data_df = None
    data_source = None

# Afficher un aperçu des données
if data_df is not None:
    print("\n" + "="*50)
    print(f"📋 Aperçu des données ({data_source}):")
    print(data_df.head(10))
    print("\n" + "="*50)
    print(data_df.info())

In [ ]:
# ============================================================
# 📊 STATISTIQUES DESCRIPTIVES
# ============================================================
if data_source == "athletes" and data_df is not None:
    print("📈 STATISTIQUES GÉNÉRALES SUR LES ATHLÈTES")
    print("="*60)
    
    print(f"\n👥 Nombre total d'athlètes : {len(data_df)}")
    
    # Analyse par genre
    if "gender" in data_df.columns:
        print("\n⚖️ Répartition par genre :")
        print(data_df["gender"].value_counts())
    
    # Analyse des médailles
    if "total_medals" in data_df.columns:
        total_medals = data_df["total_medals"].sum()
        athletes_with_medals = len(data_df[data_df["total_medals"] > 0])
        print(f"\n🏅 Total de médailles : {total_medals}")
        print(f"🏆 Athlètes médaillés : {athletes_with_medals}")
        print(f"📊 Moyenne de médailles : {data_df['total_medals'].mean():.2f}")
        
        # Top 10 athlètes
        print("\n🥇 Top 10 des athlètes les plus médaillés :")
        top_athletes = data_df.nlargest(10, "total_medals")[["full_name", "country_id", "total_medals", "gold_medals", "silver_medals", "bronze_medals"]]
        print(top_athletes.to_string(index=False))
    
    # Analyse physique
    if "height_cm" in data_df.columns and "weight_kg" in data_df.columns:
        print("\n📏 Statistiques physiques :")
        print(f"   Taille moyenne : {data_df['height_cm'].mean():.1f} cm")
        print(f"   Poids moyen : {data_df['weight_kg'].mean():.1f} kg")

elif data_source == "results" and data_df is not None:
    print("📈 STATISTIQUES SUR LES RÉSULTATS")
    print("="*60)
    
    print(f"\n📊 Nombre total de résultats : {len(data_df)}")
    
    if "rank_position" in data_df.columns:
        print("\n🏆 Statistiques des classements :")
        print(f"   Meilleure position : {data_df['rank_position'].min()}")
        print(f"   Position moyenne : {data_df['rank_position'].mean():.2f}")
        print(f"   Nombre de podiums (top 3) : {len(data_df[data_df['rank_position'] <= 3])}")
    
    if "is_record" in data_df.columns:
        records_count = data_df["is_record"].sum()
        print(f"\n🎯 Nombre de records : {records_count}")

In [ ]:
# ============================================================
# 📊 VISUALISATION 1 : Répartition par genre
# ============================================================
if data_source == "athletes" and data_df is not None and "gender" in data_df.columns:
    plt.figure(figsize=(8, 6))
    gender_counts = data_df["gender"].value_counts()
    colors = ['#3498db', '#e74c3c', '#95a5a6']
    plt.pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%', 
            colors=colors, startangle=90)
    plt.title("Répartition des Athlètes par Genre", fontsize=14, fontweight='bold')
    plt.show()

In [ ]:
# ============================================================
# 🏆 VISUALISATION 2 : Top 15 des athlètes les plus médaillés
# ============================================================
if data_source == "athletes" and data_df is not None:
    if "total_medals" in data_df.columns and "full_name" in data_df.columns:
        plt.figure(figsize=(12, 7))
        top_15 = data_df.nlargest(15, "total_medals")
        plt.barh(range(len(top_15)), top_15["total_medals"], color="steelblue")
        plt.yticks(range(len(top_15)), top_15["full_name"])
        plt.title("Top 15 des Athlètes les Plus Médaillés", fontsize=14, fontweight='bold')
        plt.xlabel("Nombre de Médailles")
        plt.ylabel("Athlète")
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.show()

In [ ]:
# ============================================================
# 🥇🥈🥉 VISUALISATION 3 : Médailles par type pour le Top 10
# ============================================================
if data_source == "athletes" and data_df is not None:
    if all(col in data_df.columns for col in ["gold_medals", "silver_medals", "bronze_medals", "full_name"]):
        top_10 = data_df.nlargest(10, "total_medals")[["full_name", "gold_medals", "silver_medals", "bronze_medals"]]
        
        fig, ax = plt.subplots(figsize=(14, 7))
        x = range(len(top_10))
        width = 0.25
        
        ax.bar([i - width for i in x], top_10["gold_medals"], width, label='Or', color='#FFD700')
        ax.bar(x, top_10["silver_medals"], width, label='Argent', color='#C0C0C0')
        ax.bar([i + width for i in x], top_10["bronze_medals"], width, label='Bronze', color='#CD7F32')
        
        ax.set_xticks(x)
        ax.set_xticklabels(top_10["full_name"], rotation=45, ha='right')
        ax.set_ylabel('Nombre de Médailles')
        ax.set_title('Répartition des Médailles - Top 10 des Athlètes', fontsize=14, fontweight='bold')
        ax.legend()
        plt.tight_layout()
        plt.show()

In [ ]:
# ============================================================
# 📏 VISUALISATION 4 : Distribution de la taille des athlètes
# ============================================================
if data_source == "athletes" and data_df is not None and "height_cm" in data_df.columns:
    plt.figure(figsize=(12, 6))
    heights = data_df["height_cm"].dropna()
    plt.hist(heights, bins=30, color="skyblue", edgecolor="black")
    plt.axvline(heights.mean(), color='red', linestyle='--', linewidth=2, label=f'Moyenne: {heights.mean():.1f} cm')
    plt.title("Distribution de la Taille des Athlètes", fontsize=14, fontweight='bold')
    plt.xlabel("Taille (cm)")
    plt.ylabel("Nombre d'Athlètes")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# ============================================================
# ⚖️ VISUALISATION 5 : Distribution du poids des athlètes
# ============================================================
if data_source == "athletes" and data_df is not None and "weight_kg" in data_df.columns:
    plt.figure(figsize=(12, 6))
    weights = data_df["weight_kg"].dropna()
    plt.hist(weights, bins=30, color="coral", edgecolor="black")
    plt.axvline(weights.mean(), color='darkred', linestyle='--', linewidth=2, label=f'Moyenne: {weights.mean():.1f} kg')
    plt.title("Distribution du Poids des Athlètes", fontsize=14, fontweight='bold')
    plt.xlabel("Poids (kg)")
    plt.ylabel("Nombre d'Athlètes")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# ============================================================
# 📊 VISUALISATION 6 : Relation Taille/Poids
# ============================================================
if data_source == "athletes" and data_df is not None:
    if "height_cm" in data_df.columns and "weight_kg" in data_df.columns:
        plt.figure(figsize=(12, 8))
        valid_data = data_df.dropna(subset=["height_cm", "weight_kg"])
        plt.scatter(valid_data["height_cm"], valid_data["weight_kg"], alpha=0.5, color="green")
        plt.title("Relation entre Taille et Poids des Athlètes", fontsize=14, fontweight='bold')
        plt.xlabel("Taille (cm)")
        plt.ylabel("Poids (kg)")
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

In [ ]:
# ============================================================
# 📊 VISUALISATION 7 : Distribution des positions (pour results)
# ============================================================
if data_source == "results" and data_df is not None and "rank_position" in data_df.columns:
    plt.figure(figsize=(12, 6))
    position_counts = data_df["rank_position"].value_counts().head(10).sort_index()
    position_counts.plot(kind="bar", color="steelblue")
    plt.title("Distribution des 10 Premières Positions", fontsize=14, fontweight='bold')
    plt.xlabel("Position")
    plt.ylabel("Nombre de Résultats")
    plt.xticks(rotation=0)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# ============================================================
# 🎯 RÉSUMÉ FINAL
# ============================================================
if data_df is not None:
    print("\n" + "=" * 60)
    print(f"           📊 RÉSUMÉ DE L'ANALYSE ({data_source.upper()}) 📊")
    print("=" * 60)
    
    if data_source == "athletes":
        print(f"\n👥 Total d'athlètes : {len(data_df):,}")
        
        if "gender" in data_df.columns:
            print(f"⚖️ Répartition par genre :")
            for gender, count in data_df["gender"].value_counts().items():
                print(f"   - {gender}: {count:,} athlètes")
        
        if "total_medals" in data_df.columns:
            print(f"\n🏅 Statistiques de médailles :")
            print(f"   - Total : {data_df['total_medals'].sum():,}")
            print(f"   - Athlètes médaillés : {len(data_df[data_df['total_medals'] > 0]):,}")
            print(f"   - Moyenne par athlète : {data_df['total_medals'].mean():.2f}")
            
            # Top 3
            print(f"\n🥇 Top 3 des athlètes :")
            top_3 = data_df.nlargest(3, "total_medals")
            for i, row in enumerate(top_3.itertuples(), 1):
                print(f"   {i}. {row.full_name} : {row.total_medals} médailles")
        
        if "height_cm" in data_df.columns and "weight_kg" in data_df.columns:
            print(f"\n📏 Caractéristiques physiques moyennes :")
            print(f"   - Taille : {data_df['height_cm'].mean():.1f} cm")
            print(f"   - Poids : {data_df['weight_kg'].mean():.1f} kg")
    
    elif data_source == "results":
        print(f"\n📊 Total de résultats : {len(data_df):,}")
        
        if "rank_position" in data_df.columns:
            print(f"\n🏆 Statistiques de classement :")
            print(f"   - Podiums (top 3) : {len(data_df[data_df['rank_position'] <= 3]):,}")
            print(f"   - Position moyenne : {data_df['rank_position'].mean():.2f}")
        
        if "is_record" in data_df.columns:
            records = data_df["is_record"].sum()
            print(f"\n🎯 Records établis : {records}")
    
    print("\n" + "=" * 60)
else:
    print("\n⚠️ Aucune donnée disponible pour générer un résumé.")